## K-Nearest Neighbors  

### Objetivos  

Después de completar este cuaderno, serás capaz de:  

* Utilizar **K-Nearest Neighbors** para clasificar datos  


En este cuaderno, cargarás un conjunto de datos de clientes, ajustarás los datos y utilizarás **K-Nearest Neighbors** para predecir un punto de datos. Pero, ¿qué es **K-Nearest Neighbors**?  


**K-Nearest Neighbors** es un algoritmo de aprendizaje supervisado donde los datos se "entrenan" con puntos de datos correspondientes a su clasificación. Para predecir la clase de un punto de datos dado, el algoritmo toma en cuenta las clases de los **K** puntos de datos más cercanos y elige la clase a la que pertenece la mayoría de los **K** vecinos más cercanos como la clase predicha. 


### Aquí hay una visualización del algoritmo K-Nearest Neighbors.  

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%203/images/KNN_Diagram.png">  


En este caso, tenemos puntos de datos de la Clase A y B. Queremos predecir a qué clase pertenece la estrella (punto de prueba). Si consideramos un valor de **K=3** (3 puntos de datos más cercanos), obtendremos una predicción de la **Clase B**. Sin embargo, si consideramos un valor de **K=6**, obtendremos una predicción de la **Clase A**.  


En este sentido, es importante considerar el valor de **K**. A partir de este diagrama, deberías tener una idea de cómo funciona el algoritmo **K-Nearest Neighbors**. Este considera los **K** vecinos más cercanos (puntos de datos) al predecir la clasificación del punto de prueba.  


<h1>Tabla de contenido</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#About-the-dataset">Sobre el conjunto de datos</a></li>
        <li><a href="#Data-Visualization-and-Analysis">Visualización y análisis de datos</a></li>
        <li><a href="#classification">Clasificación</a></li>
    </ol>
</div>
<br>
<hr>



### Cargar las librerías necesarias  


In [ ]:
!pip install scikit-learn
!pip install matplotlib
!pip install pandas 
!pip install numpy 
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
from sklearn import preprocessing
import pandas as pd
import numpy as np

<div id="about_dataset">
    <h2>Sobre el conjunto de datos</h2>
</div>


Imagina que un proveedor de telecomunicaciones ha segmentado su base de clientes según patrones de uso del servicio, categorizando a los clientes en cuatro grupos. Si los datos demográficos pueden usarse para predecir la pertenencia a un grupo, la empresa puede personalizar ofertas para clientes potenciales individuales. Este es un problema de clasificación. Es decir, dado el conjunto de datos con etiquetas predefinidas, necesitamos construir un modelo para predecir la clase de un caso nuevo o desconocido.  

El ejemplo se enfoca en utilizar datos demográficos, como región, edad y estado civil, para predecir patrones de uso.  

El campo objetivo, llamado **custcat**, tiene cuatro posibles valores que corresponden a los cuatro grupos de clientes:  
1- Basic Service  
2- E-Service  
3- Plus Service  
4- Total Service  

Nuestro objetivo es construir un clasificador para predecir la clase de casos desconocidos. Usaremos un tipo específico de clasificación llamado **K-Nearest Neighbors**.  


### Cargar los datos


Leamos los datos usando la biblioteca **pandas** e imprimamos las primeras cinco filas. 


In [ ]:
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%203/data/teleCust1000t.csv')
df.head()

<div id="visualization_analysis">
    <h2>Visualización y análisis de datos</h2> 
</div>


#### Veamos cuántos clientes pertenecen a cada clase en nuestro conjunto de datos  


In [ ]:
df['custcat'].value_counts()

#### 281 Plus Service, 266 Basic-service, 236 Total Service, 17 E-Service customers


Podemos explorar nuestros datos utilizando técnicas de visualización:  


In [ ]:
df.hist(column='income', bins=50)

### Definir el conjunto de características  


Definamos el conjunto de características, **X**:  

In [ ]:
df.columns


Para utilizar la biblioteca **scikit-learn**, debemos convertir el **DataFrame de Pandas** en un **array de Numpy**:  

In [ ]:
X = df[['region', 'tenure','age', 'marital', 'address', 'income', 'ed', 'employ','retire', 'gender', 'reside']] .values  #.astype(float)
X[0:5]


¿Cuáles son nuestras etiquetas?  


In [ ]:
y = df['custcat'].values
y[0:5]

### Normalizando de datos

La **estandarización de datos** ajusta los valores para que tengan **media cero y varianza unitaria**. Esto es una buena práctica, especialmente para algoritmos como **KNN**, que se basan en la distancia entre puntos de datos:  


In [ ]:
X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))
X[0:5]

### División en entrenamiento y prueba (Train/Test Split)  

La **precisión fuera de muestra** (**Out of Sample Accuracy**) es el porcentaje de predicciones correctas que el modelo realiza sobre datos con los que **NO ha sido entrenado**. Si realizamos el entrenamiento y la prueba en el mismo conjunto de datos, es probable que la precisión fuera de muestra sea baja, debido a la posibilidad de **sobreajuste** (**overfitting**) del modelo.  

Es fundamental que nuestros modelos tengan una alta **precisión fuera de muestra**, ya que el objetivo de cualquier modelo es hacer predicciones correctas sobre datos desconocidos. ¿Cómo podemos mejorar la precisión fuera de muestra? Una forma es utilizar un enfoque de evaluación llamado **Train/Test Split**.  

La **división en entrenamiento y prueba** consiste en dividir el conjunto de datos en **dos subconjuntos**:  
- **Conjunto de entrenamiento** (**Training set**)  
- **Conjunto de prueba** (**Testing set**)  

Estos conjuntos son **mutuamente excluyentes**. Primero, el modelo se **entrena** con el conjunto de entrenamiento y luego se **evalúa** con el conjunto de prueba.  

Este método proporciona una evaluación más precisa de la **precisión fuera de muestra**, ya que el conjunto de prueba no forma parte de los datos utilizados para entrenar el modelo. Esto lo hace **más realista para problemas del mundo real**.


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

<div id="classification">
    <h2>Clasificación</h2>
</div>


<h3>K nearest neighbor (KNN)</h3>


#### Importar la librería


El clasificador implementa el método de votación de los **K vecinos más cercanos**.  


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

### Entrenamiento

Empezamos el algoritmo con k=4:


In [ ]:
k = 4  
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
neigh

### Predicción

Podemos usar el modelo para hacer predicciones en el conjunto de prueba:  


In [ ]:
yhat = neigh.predict(X_test)
yhat[0:5]

### Evaluación de exactitud

En la clasificación multietiqueta, el puntaje de exactitud de clasificación (accuracy classification score) es una función que calcula la exactitud por subconjunto. Esta función es equivalente a la función `jaccard_score`. En esencia, mide qué tan bien coinciden las etiquetas reales y las etiquetas predichas en el conjunto de prueba.


In [ ]:
from sklearn import metrics
print("Exactitud del conjunto de entrenamiento: ", metrics.accuracy_score(y_train, neigh.predict(X_train)))
print("Exactitud del conjunto de prueba: ", metrics.accuracy_score(y_test, yhat))

### Práctica  

¿Puedes construir el modelo nuevamente, pero esta vez con **k=6**?


In [ ]:
# Escribe tu codigo aqui




<details><summary>Click para la solución</summary>

```python
k = 6
neigh6 = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
yhat6 = neigh6.predict(X_test)
print("Exactitud del conjunto de entrenamiento: ", metrics.accuracy_score(y_train, neigh6.predict(X_train)))
print("Exactitud del conjunto de prueba: ", metrics.accuracy_score(y_test, yhat6))

```

</details>


#### ¿Qué pasa con otros valores de K?  

En **KNN**, **K** es el número de vecinos más cercanos a examinar. Este valor debe ser especificado por el usuario. Entonces, ¿cómo podemos elegir el valor correcto de **K**?  

La solución general es **reservar una parte de los datos para probar la precisión del modelo**. Luego, se elige **K = 1**, se usa el conjunto de entrenamiento para modelar y se calcula la precisión de la predicción usando todas las muestras del conjunto de prueba.  

Este proceso se repite aumentando **K**, y se observa cuál es el mejor valor de **K** para el modelo.  

Podemos calcular la exactitud de **KNN** para diferentes valores de **K**.


In [ ]:
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))

for n in range(1,Ks):  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

mean_acc

#### Graficar la exactitud del modelo para diferentes valores de vecinos.


In [ ]:
plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.fill_between(range(1,Ks),mean_acc - 3 * std_acc,mean_acc + 3 * std_acc, alpha=0.10,color="green")
plt.legend(('Exactitud ', '+/- 1xstd','+/- 3xstd'))
plt.ylabel('Exactitud ')
plt.xlabel('Numero de vecinos (K)')
plt.tight_layout()
plt.show()

In [ ]:
print( "La mejor exactitud fue con: ", mean_acc.max(), "with k=", mean_acc.argmax()+1) 